## Tests

Test: check that the gb genebank file info matches a fafsa file sequence. 
A 'diff'.

python3 -m venv .venv-dependencies
source .venv-dependencies/bin/activate
pip install -r requirements.txt
python -m ipykernel install --user \
    --name dependencies-venv \
    --display-name "Python (.venv-dependencies)"


In [ ]:
## %run ./env_check.py

In [1]:
# Run this code inside a Jupyter cell. If all three lines point to your .venv folder in WSL, your setup is perfect:

import sys
import os

print(f"1. Executable: {sys.executable}") 
print(f"2. Version: {sys.version}")
print(f"3. Env Path: {os.getenv('VIRTUAL_ENV')}")

1. Executable: /home/morgan/projects/bioinfo-projects/comparative-sequence-analysis/.venv/bin/python
2. Version: 3.11.2 (main, Apr 28 2025, 14:11:48) [GCC 12.2.0]
3. Env Path: /home/morgan/projects/bioinfo-projects/comparative-sequence-analysis/.venv


In [2]:
from os import path

In [55]:
import Bio
from Bio import SeqIO


In [ ]:
from Bio import GenBank
def get_gb_sequence__orig(g_path):
    with open(g_path, 'r') as G_file: 
        records = [x for  x in GenBank.parse(G_file)]
        if not len(records)==1:
           raise RuntimeError(f"Number of records not handled: {str(len(records))}")
        return records[0].sequence

In [54]:
def get_gb_sequence(g_path):
    for ind,seq_record in enumerate(SeqIO.parse(g_path, "gb")):
        print('--- seq number', ind)
        print(seq_record.id)
        print(repr(seq_record.seq))
        print(len(seq_record))
    return seq_record.seq

In [57]:
def get_fasta_sequence(f_path):
    for ind,seq_record in enumerate(SeqIO.parse(f_path, "fasta")):
        print('--- seq number', ind)
        print(seq_record.id)
        print(repr(seq_record.seq))
        print(len(seq_record))
    return seq_record.seq

In [63]:
def verify_gb_fafsa_compatibility(g, f, relpath=''):
    g_path = path.join(relpath, g)
    f_path = path.join(relpath, f)
    print('gb:')
    gb_seq = get_gb_sequence(g_path)
    print('fasta:')
    fa_seq = get_fasta_sequence(f_path)
    print('done')
    return gb_seq, fa_seq


In [64]:
gb_seq, fa_seq = verify_gb_fafsa_compatibility('sequence.gb', 'ncbi_dataset/data/gene.fna', relpath='./data/sirt1-human')

gb:
--- seq number 0
NM_001142498.2
Seq('GCATCTCCTCCTCCCTCTCCCCGGGCTCCTACTGGCCTGAGGTTGAGGGCGGCT...TTA')
3588
fasta:
--- seq number 0
NC_000010.11:67884656-67918390
Seq('GCCAGTGCCGCGCGTCGAGCGGGAGCAGAGGAGGCGAGGGAGGAGGGCCAGAGA...TTA')
33735
--- seq number 1
NC_060934.1:68753401-68787131
Seq('GCCAGTGCCGCGCGTCGAGCGGGAGCAGAGGAGGCGAGGGAGGAGGGCCAGAGA...TTA')
33731
done


In [69]:
print(gb_seq[:120])
print(fa_seq[25:][:120])


GCATCTCCTCCTCCCTCTCCCCGGGCTCCTACTGGCCTGAGGTTGAGGGCGGCTGGGGGCTCGGGGCAGGCTCCGCGGCGTTCCCCTCCCCACCCCGGCCCTCCGTTCAGCCGCGCTCCT
CAGAGGAGGCGAGGGAGGAGGGCCAGAGAGGCAGTTGGAAGATGGCGGACGAGGCGGCCCTCGCCCTTCAGCCCGGCGGCTCCCCCTCGGCGGCGGGGGCCGACAGGGAGGCCGCGTCGT
